In [1]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets

democracy = pd.read_csv('https://raw.githubusercontent.com/DanielStuiver/DEZEWERKT/master/vrijheid.csv', low_memory=False)
happiness = pd.read_csv('https://raw.githubusercontent.com/DanielStuiver/DEZEWERKT/master/blijheid.csv', low_memory=False)

minus_jaar_vrijheid = democracy[(democracy['ID_year'] >= 2013) & (democracy['ID_year'] <= 2020) & (democracy['ID_year'] != 2014)]

minus_jaar_blijheid = happiness[(happiness['Year'] >= 2013) & (happiness['Year'] <= 2020)]

merged_data = pd.merge(minus_jaar_blijheid, minus_jaar_vrijheid, left_on=['Year', 'Country'], right_on=['ID_year', 'ID_country_name'])

df = merged_data.drop('ID_year', axis=1)
df = merged_data.drop('ID_country_name', axis=1)

#######################################################################################################################

landcodes_df = pd.read_csv('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv')
gegevens_2020 = df[df['Year'] == 2020]
gegevens_2013 = df[df['Year']== 2013]
landcodes_df['name'] = landcodes_df['name'].replace({'United States of America' : 'United States',
                                                   'Iran (Islamic Republic of)': 'Iran',
                                                   'Moldova, Republic of':'Moldova',
                                                    'Russian Federation': 'Russia',
                                                    'Taiwan, Province of China': 'Taiwan',
                                                    'Tanzania, United Republic of': 'Tanzania',
                                                    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
                                                    'Venezuela (Bolivarian Republic of)': 'Venezuela'})

gegevens_2020_mix = pd.merge(gegevens_2020, landcodes_df, left_on='Country', right_on='name', how='left',indicator=True)
gegevens_2013_mix = pd.merge(gegevens_2013, landcodes_df, left_on='Country', right_on='name', how='left',indicator=True)

different_countries = gegevens_2020_mix[gegevens_2020_mix['_merge'] != 'both']

#######################################################################################################################

fig1 = go.Figure(data=go.Choropleth(
    locations = gegevens_2020_mix['alpha-3'],
    z = gegevens_2020_mix['Index'],
    text = gegevens_2020_mix['Country'],
    colorscale = 'Inferno',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Happiness'))

fig2 = go.Figure(data=go.Choropleth(
    locations = gegevens_2013_mix['alpha-3'],
    z = gegevens_2013_mix['Index'],
    text = gegevens_2013_mix['Country'],
    colorscale = 'Inferno',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_title = 'Happiness'))

fig1.update_layout(title = 'Happiness-score landen 2020',
                   annotations = [dict(xref='paper',
                   yref='paper',
                   x=0.5, y=-0.25,
                   showarrow=False,
                   text ='Deze kaart weergeeft de happiness index-score van de landen opgenomen in het onderzoek van de VN in 2020')])
fig2.update_layout(title = 'Happiness-score landen 2013',
                   annotations = [dict(xref='paper',
                   yref='paper',
                   x=0.5, y=-0.25,
                   showarrow=False,
                   text ='Deze kaart weergeeft de happiness index-score van de landen opgenomen in het onderzoek van de VN in 2013')])

#######################################################################################################################

data = {
    'Years': ['2013', '2015', '2016', '2017', '2018', '2019', '2020'],
    'Mid-range performing democracy': [54, 52, 49, 51, 55, 56, 52],
    'Authoritarian Regime': [26, 28, 30, 31, 35, 34, 30],
    'Weak democracy': [22, 23, 22, 21, 20, 19, 21],
    'Hybrid Regime': [23, 21, 21, 22, 17, 18, 20],
    'High performing democracy': [17, 18, 19, 18, 17, 16, 17]
}

df2 = pd.DataFrame(data)

barplot = go.Figure()
barplot.add_trace(go.Bar(x=df2['Years'], y=df2['High performing democracy'], name='High performing democracy', marker = dict(color = 'rgb(255, 255, 0)')))
barplot.add_trace(go.Bar(x=df2['Years'], y=df2['Mid-range performing democracy'], name='Mid-range performing democracy', marker = dict(color='rgb(102,194,165)') )) 
barplot.add_trace(go.Bar(x=df2['Years'], y=df2['Weak democracy'], name='Weak democracy', marker = dict(color = 'rgb(141,160,203)')))
barplot.add_trace(go.Bar(x=df2['Years'], y=df2['Hybrid Regime'], name='Hybrid Regime', marker = dict(color = 'rgb(255, 192, 203)')))
barplot.add_trace(go.Bar(x=df2['Years'], y=df2['Authoritarian Regime'], name='Authoritarian Regime', marker = dict(color = 'rgb(252,141,98)')))

layout = go.Layout(title='Verschillende voorkomende vormen van democratie in de wereld', 
                   xaxis_title = 'Jaren', 
                   yaxis_title = 'Aantal landen',
                   annotations = [dict(xref='paper',
                   yref='paper',
                   x=0.5, y=-0.25,
                   showarrow=False,
                   text ='Deze grafiek visualiseert hoeveel landen deel uit maken van elke categorie van democratie')])

# barplot.update_layout(layout)

#######################################################################################################################

countries = ['Iceland', 'Venezuela', 'Romania']

country_dropdown = widgets.Dropdown(
    description='Land:',
    options=countries,
    value="Iceland")

container = widgets.VBox(children=[country_dropdown])

def update_plots(change):
    selected_country = change.new
    
    fig_widget.data = []
    
    if selected_country == 'Iceland':
        country_data = df[df['Country'] == 'Iceland'].copy()
        country_data.loc[:, 'v_41_01'] = country_data['v_41_01'] * 10
        country_data = country_data.rename(columns={'Index': 'Happiness index', 'v_41_01': 'Score stelling'})
        fig = px.line(country_data, x='Year', y=['Happiness index', 'Score stelling'], markers=True)
        for trace in fig.data:
            fig_widget.add_trace(trace)
    elif selected_country == 'Venezuela':
        country_data = df[df['Country'] == 'Venezuela'].copy()
        country_data.loc[:, 'v_41_01'] = country_data['v_41_01'] * 10
        country_data = country_data.rename(columns={'Index': 'Happiness index', 'v_41_01': 'Score stelling'})
        fig = px.line(country_data, x='Year', y=['Happiness index', 'Score stelling'], markers=True)
        for trace in fig.data:
            fig_widget.add_trace(trace)
    elif selected_country == 'Romania':
        country_data = df[df['Country'] == 'Romania'].copy()
        country_data.loc[:, 'v_41_01'] = (country_data['v_41_01'] * 10)
        country_data = country_data.rename(columns={'Index': 'Happiness index', 'v_41_01': 'Score stelling'})
        fig = px.line(country_data, x='Year', y=['Happiness index', 'Score stelling'], markers=True)
        for trace in fig.data:
            fig_widget.add_trace(trace)

country_dropdown.observe(update_plots, names='value')

fig_widget = go.FigureWidget()

fig_widget.update_layout(title = 'Hoe vaak verlenen werknemers in de publieke sector gunsten in ruil voor omkopingen of andere materiële wensen?',
                          title_font_size=13,
                          xaxis_title='Jaar',
                          yaxis_title='Score',
                          annotations = [dict(xref='paper',
                                        yref='paper',
                                        x=0.5, y=-0.25,
                                        showarrow=False,
                                        text ='Deze Grafiek toont de relatie aan tussen de happiness index en de algemene mate van corruptie in een land')])


# display(container, fig_widget)

#######################################################################################################################

countries = ['Iceland', 'Venezuela', 'Romania']

country_dropdown = widgets.Dropdown(
    description='Land:',
    options=countries)

container2 = widgets.VBox(children=[country_dropdown])

def update_plots(change):
    selected_country = change.new
    
    fig_widget2.data = []
    
    if selected_country == 'Iceland':
        country_data = df[df['Country'] == 'Iceland'].copy()
        country_data.loc[:, 'v_23_01'] = country_data['v_23_01'] * 10
        country_data = country_data.rename(columns={'Index': 'Happiness index', 'v_23_01': 'Score stelling'})
        fig = px.line(country_data, x='Year', y=['Happiness index', 'Score stelling'], markers=True)
        for trace in fig.data:
            fig_widget2.add_trace(trace)
    elif selected_country == 'Venezuela':
        country_data = df[df['Country'] == 'Venezuela'].copy()
        country_data.loc[:, 'v_23_01'] = country_data['v_23_01'] * 10
        country_data = country_data.rename(columns={'Index': 'Happiness index', 'v_23_01': 'Score stelling'})
        fig = px.line(country_data, x='Year', y=['Happiness index', 'Score stelling'], markers=True)
        for trace in fig.data:
            fig_widget2.add_trace(trace)
    elif selected_country == 'Romania':
        country_data = df[df['Country'] == 'Romania'].copy()
        country_data.loc[:, 'v_23_01'] = (country_data['v_23_01'] * 10)
        country_data = country_data.rename(columns={'Index': 'Happiness index', 'v_23_01': 'Score stelling'})
        fig = px.line(country_data, x='Year', y=['Happiness index', 'Score stelling'], markers=True)
        for trace in fig.data:
            fig_widget2.add_trace(trace)

country_dropdown.observe(update_plots, names='value')

fig_widget2 = go.FigureWidget()

fig_widget2.update_layout(title='Genieten arme mensen evenveel van burgerlijke vrijheden als rijke mensen?',
                          title_font_size=13,
                          xaxis_title='Jaar',
                          yaxis_title='Score',
                          annotations = [dict(xref='paper',
                                        yref='paper',
                                        x=0.5, y=-0.25,
                                        showarrow=False,
                                        text ='Deze Grafiek toont de relatie aan tussen de happiness index en de algemene mate van gelijkheid in een land')])

# display(container2, fig_widget2)

#######################################################################################################################

countries = ['Iceland', 'Venezuela', 'Romania']

country_dropdown = widgets.Dropdown(
    description='Land:',
    options=countries)

container3 = widgets.VBox(children=[country_dropdown])

def update_plots(change):
    selected_country = change.new
    
    fig_widget3.data = []
    
    if selected_country == 'Iceland':
        country_data = df[df['Country'] == 'Iceland'].copy()
        country_data.loc[:, 'v_22_01'] = country_data['v_22_01'] * 10
        country_data = country_data.rename(columns={'Index': 'Happiness index', 'v_22_01': 'Score stelling'})
        fig = px.line(country_data, x='Year', y=['Happiness index', 'Score stelling'], markers=True)
        for trace in fig.data:
            fig_widget3.add_trace(trace)
    elif selected_country == 'Venezuela':
        country_data = df[df['Country'] == 'Venezuela'].copy()
        country_data.loc[:, 'v_22_01'] = country_data['v_22_01'] * 10
        country_data = country_data.rename(columns={'Index': 'Happiness index', 'v_22_01': 'Score stelling'})
        fig = px.line(country_data, x='Year', y=['Happiness index', 'Score stelling'], markers=True)
        for trace in fig.data:
            fig_widget3.add_trace(trace)
    elif selected_country == 'Romania':
        country_data = df[df['Country'] == 'Romania'].copy()
        country_data.loc[:, 'v_22_01'] = (country_data['v_22_01'] * 10)
        country_data = country_data.rename(columns={'Index': 'Happiness index', 'v_22_01': 'Score stelling'})
        fig = px.line(country_data, x='Year', y=['Happiness index', 'Score stelling'], markers=True)
        for trace in fig.data:
            fig_widget3.add_trace(trace)

country_dropdown.observe(update_plots, names='value')

fig_widget3 = go.FigureWidget()

fig_widget3.update_layout(title='Probeert de regering op directe of indirecte wijze de gedrukte of uitzendmedia te censureren?',
                          title_font_size=13,
                          xaxis_title='Jaar',
                          yaxis_title='Score',
                          annotations = [dict(xref='paper',
                                        yref='paper',
                                        x=0.5, y=-0.25,
                                        showarrow=False,
                                        text ='Deze Grafiek toont de relatie aan tussen de happiness index en de algemene mate van censuur in een land')])

#######################################################################################################################


countries = ['Iceland', 'Venezuela', 'Romania']
colors = ['rgb(100,40,255)', 'rgb(102,194,165)', 'rgb(252,141,98)']

fig_widget4 = go.Figure()

# Line for Iceland
country_data_iceland = df[df['Country'] == 'Iceland'].copy()
country_data_iceland = country_data_iceland.rename(columns={'Index': 'Happiness index'})
fig_iceland = go.Scatter(
    x=country_data_iceland['Year'],
    y=country_data_iceland['Happiness index'],
    mode='lines',
    marker=dict(color=colors[0]),
    name=countries[0]
)
fig_widget4.add_trace(fig_iceland)

# Line for Venezuela
country_data_venezuela = df[df['Country'] == 'Venezuela'].copy()
country_data_venezuela = country_data_venezuela.rename(columns={'Index': 'Happiness index'})
fig_venezuela = go.Scatter(
    x=country_data_venezuela['Year'],
    y=country_data_venezuela['Happiness index'],
    mode='lines',
    marker=dict(color=colors[1]),
    name=countries[1]
)
fig_widget4.add_trace(fig_venezuela)

# Line for Romania
country_data_romania = df[df['Country'] == 'Romania'].copy()
country_data_romania = country_data_romania.rename(columns={'Index': 'Happiness index'})
fig_romania = go.Scatter(
    x=country_data_romania['Year'],
    y=country_data_romania['Happiness index'],
    mode='lines',
    marker=dict(color=colors[2]),
    name=countries[2]
)
fig_widget4.add_trace(fig_romania)

fig_widget4.update_layout(
    title='Happiness index voor Ijsland, Venezuela en Roemenië',
    title_font_size=13,
    xaxis_title='Jaar',
    yaxis_title='Happiness Index',
    annotations=[dict(
        xref='paper',
        yref='paper',
        x=0.5,
        y=-0.25,
        showarrow=False,
        text='Deze Grafiek toont de happiness index aan voor de drie gekozen landen'
    )]
)

print()
# display(container3, fig_widget3)

/Users/daniel/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# Inleiding 


In een wereld waarin democratie en geluk beide belangrijke maatschappelijke doelen vertegenwoordigen, is het interessant om te onderzoeken hoe deze twee concepten met elkaar verbonden zijn. Democratie wordt vaak gezien als een middel om het welzijn en de vrijheid van mensen te bevorderen, terwijl geluk wordt beschouwd als een ultieme indicator van persoonlijk welzijn en tevredenheid. Deze datastory, heeft als doel de democracy index en de happiness index te visualiseren, om zo inzicht te bieden in de mogelijke relatie tussen democratie en geluk.

Het voornaamste doel van dit project is het creëren van visuele representaties van de democracy index en de happiness index. De democracy index is een bekende maatstaf voor de staat van de democratie in landen over de hele wereld, terwijl de happiness index meet hoe tevreden en gelukkig mensen zich voelen in verschillende landen. Door deze twee indexen te combineren en te visualiseren, hopen we een beter begrip te krijgen van de mogelijke verbanden en patronen tussen democratie en geluk. De vraag die we stellen is dan ook: bevordert democratie voor geluk?

Deze vraag zal beantwoord worden door de datasets samen te voegen en op een verkennende manier te onderzoeken. Wellicht wordt er een link gevonden tussen de vrijheid van een land en de tevredenheid van haar burgers, maar het tegendeel zou ook ondervonden kunnen worden, waaruit blijkt dat mensen gelukkig of ongelukkig kunnen zijn onafhankelijk van de staat waarin hun overheid zich verkeert.

## Datasets and preprocessing - Happiness index

De happiness index is een maatstaf die bedacht is voor de World Happiness Report, een rapportage die vanaf 2012 regelmatig door de Sustainable Development Solutions Network, een initiatief van de Verenigde Naties, wordt geschreven. Het initiële doel van het rapportage was om het perspectief van mensen over hun eigen geluk te meten, sindsdien wordt het rapportage elk jaar opnieuw uitgebracht. De happiness index is een vrij eenvoudige maatstaf die bepaald wordt door een gemiddelde van antwoorden geworven van mensen middels het onderzoeksbureau Gallup. Mensen worden gevraagd om een ladder voor te stellen, waarin een 0 het slechtst mogelijke leven voorstelt, en een 10 het best mogelijke leven, daarna worden de mensen gevraagd waar ze zichzelf op deze ladder zetten.

De dataset die gebruikt zal worden voor dit onderzoek is als volgt:

https://www.kaggle.com/datasets/simonaasm/world-happiness-index-by-reports-2013-2023

Deze dataset bevat de happiness index voor 167 landen voor de periode van 2013 tot en met 2023. Het eerste rapportage uit 2012 is niet opgenomen in deze dataset, ook ontbreken er waardes voor 2014, aangezien er geen rapportage werd opgesteld in dat jaar. Naast de naam van elk land en het jaar van het datapunt zijn er in de dataset nog twee variabelen; dit zijn de happiness index met maximaal 3 decimalen na de punt, en de rang van het land in dat jaar vergeleken met de andere landen.

In [2]:
fig2.show()
#In de onderstaande afbeelding is per land te zien wat de gemiddelde happiness score is in 2013.

In [3]:
fig1.show()
#In de onderstaande afbeelding is per land te zien wat de gemiddelde happiness score is in 2020.

## Datasets and Preprocessing - The Global State of Democracy Indices


The Global State of Democracy Indices is een uitgebreide evaluatie van verschillende factoren die de mate van democratie in een land proberen te meten, opgesteld door het International Institute for Democracy and Electoral Assistance. Doordat een definitie van democratie niet universeel en ook niet gemakkelijk te bepalen is, bestaat het rapportage uit veel verschillende onderwerpen, waarbij de 174 gemeten landen een score krijgen voor elk van deze onderwerpen tussen 0 en 1, met 0 het meest negatief, en 1 het meest positief. Deze scores worden gemeten aan de hand van onderzoek, de ervaringen van burgers en de meningen van journalisten.

De dataset die gebruikt zal worden voor ons onderzoek is als volgt:

https://www.kaggle.com/datasets/aliaamiri/the-global-state-of-democracy-indices

Deze dataset geeft de scores voor elk land in elk van de 157 kwesties tussen 1975 en 2020, deze kwesties zijn de observeerbare variabelen in deze dataset. De namen van de variabelen worden gebruikt om elke kwestie te categoriseren, de uitleg voor elke kwestie staat in een pdf bestand dat meegeleverd werd met de dataset. Elke kwestie beslaat een vraag of stelling die een indicator kan zijn van de mate van democratie in een land. Kwesties kunnen onderwerpen aanslaan als gelijke rechten, persvrijheid, en censuur.

In [4]:
barplot.update_layout(layout)
##In de onderstaande afbeelding zijn de voorkomnde vormen van democratie in de wereld weergeven in aantallen. 

### Creëren van de uiteindelijke dataset

Om een dataset te krijgen die de gegevens van de twee gevonden datasets combineert, moesten de variabelen op elkaar aan kunnen sluiten, dit is gedaan middels de merge functie van Pandas. Eerst werd de gecombineerde dataset opgesteld door de twee datasets samen te voegen op jaartal, waardoor de uiteindelijke dataset gegevens zou hebben van 2013 tot en met 2020 (m.u.v 2014). Daarnaast werden de datasets gecombineerd op land, aangezien beide datasets niet evenveel landen bevatten. Hiervoor moesten tevens de namen van een aantal landen aangepast worden, zodat deze universeel waren.

# Perspectieven

Voor dit onderzoek zal er vanuit twee tegenovergestelde perspectieven bekeken worden naar de kwestie of er een verband bestaat tussen de mate van democratie en het algemene niveau van geluk binnen een land.

Het eerste perspectief zal beargumenteren dat een hogere mate van democratie, ook zal leiden tot een hogere ervaring van geluk, met als hoofdargument dat democratie vanzelfsprekend positief is, en daardoor haar burgers positieve gevoelens mee te brengen door middel van de mogelijkheid van bevolkingsparticipatie en het gevoel dat de overheid naar haar burgers luistert.

Het andere perspectief zal beargumenteren dat, los van de discussie of democratie een positieve of negatieve kracht is, geen primaire bron van geluk is voor de inwoners van een land. Het voornaamste argument voor dit perspectief zal zijn dat er veel dingen zijn die mensen geluk bezorgen, maar de mate van democratie niet één van de grootste is, en daardoor weinig overeen zal komen met metingen van geluk.

# Observaties

Om te ondervinden of er een correlatie bestaat tussen de mate van democratie en de mate van geluk in een land, is het belangrijk om te weten wat er geobserveerd moet worden. Om het onderzoek een schaal te geven die behandelbaar was, is er voor gekozen 3 metingen van democratie te selecteren. Voor elk van deze drie metingen wordt er gekeken naar drie landen. 

De Global State of Democracy Indices dataset bevat meer dan 160 verschillende metingen van democratie, deze metingen spannen over meerdere onderwerpen die bij elkaar een basis vormen die toe staat te monitoren hoe het met de democratie gesteld is in een land. Elke meting in de dataset is één voor één bekeken, en de bredere metingen met potentie werden genoteerd. Uiteindelijk zijn er drie geselecteerd die de hoofdonderwerpen goed vertegenwoordigen:

<b>Origineel</b>: How routinely do public sector employees grant favours in exchange for bribes or other material inducements?
<br><b>Nederlands</b>: Hoe vaak verlenen werknemers in de publieke sector gunsten in ruil voor omkopingen of andere materiële wensen?
<br><b>Variabele</b>: V_41_01
<br><b>Hoofdonderwerp</b>: Corruptie

<b>Origineel</b>: Do poor people enjoy the same level of civil liberties as rich people do?
<br><b>Nederlands</b>: Genieten arme mensen evenveel van burgerlijke vrijheden als rijke mensen?
<br><b>Variabele</b>: V_23_01
<br><b>Hoofdonderwerp</b>: Gelijkheid

<b>Origineel</b>: Does the government directly or indirectly attempt to censor the print or broadcast media?
<br><b>Nederlands</b>: Probeert de regering op directe of indirecte wijze de gedrukte of uitzendmedia te censureren?
<br><b>Variabele</b>:V_22_01
<br><b>Hoofdonderwerp</b>: Censuur

De drie gekozen landen zijn gekozen om ieders twee verschillende variabelen te representeren; de mate van welvaart en de verandering van de happiness index. 

Ijsland is gekozen als centrumland met een stabiele happiness index

Roemenië is gekozen als semi-periferisch land met een stijgende happiness index

Venezuela is gekozen als periferisch land met een dalende happiness index

In [5]:
fig_widget4.show()

### Belangrijke opmerkingen met betrekking tot de verwerkte data

Er is geen data voor het jaar 2014, dit is vanwege het feit dat er in dat jaar geen World Happiness Report is geschreven door de VN. 

De score voor de Democracy Indices is op een schaal van 0 tot 1, met 0 als slechtst mogelijke situatie, en 1 de best mogelijke situatie. De score van de happiness index staan op een schaal van 0 tot 10, met 0 als slechtst mogelijke situatie en 10 als best mogelijke situatie. Om de scores makkelijk te kunnen vergelijken zijn de waardes van de democracy indices vermedigvuldigt met een factor van 10.

In [6]:
display(container, fig_widget)


FigureWidget({
    'data': [],
    'layout': {'annotations': [{'showarrow': False,
                                'text': ('Deze Grafiek toont de relatie ' ... 'mate van corruptie in een land'),
                                'x': 0.5,
                                'xref': 'paper',
                                'y': -0.25,
                                'yref': 'paper'}],
               'template': '...',
               'title': {'font': {'size': 13},
                         'text': ('Hoe vaak verlenen werknemers i' ... 'en of andere materiële wensen?')},
               'xaxis': {'title': {'text': 'Jaar'}},
               'yaxis': {'title': {'text': 'Score'}}}
})

## Analyse van stelling: Corruptie

Bovenstaande grafiek visualiseert de happiness index met de stelling: 'Hoe vaak verlenen werknemers in de publieke sector gunsten in ruil voor omkopingen of andere materiële wensen?'. IJsland scoort relatief erg hoog met betrekking tot de stelling. Werknemers worden dus relatief weinig omgekocht, dit heeft een positieve invloed op de democracy index. Ook is te zien dat de score van IJsland niet fluctueert, waaruit ook blijkt dat het land een sterke democratie is. De happiness index van IJsland is ook relatief sterk, deze schommelt rond de 7.5. 

Venezuela toont een geheel andere grafiek. De score m.b.t. de stelling is extreem laag. In 2013 scoort het land nog rond de 1, maar dit zakt al snel af richting 0.2. Dit gebeurt in 2015. In datzelfde jaar neemt ook de happiness index af. De Pearson Correlation Coefficient (PCC) tussen de twee scores is 0.788. Er is dus een correlatie. 

Roemenië heeft een stijgende happiness index. Het land wordt gedurende de jaren blijer. Van 2016-2017 is er een flinke daling te zien in de score m.b.t. de stelling. De lage PCC van 0.180 zegt echter dat er geen verband is tussen de twee.

In [7]:
display(container2, fig_widget2)


FigureWidget({
    'data': [],
    'layout': {'annotations': [{'showarrow': False,
                                'text': ('Deze Grafiek toont de relatie ' ... 'ate van gelijkheid in een land'),
                                'x': 0.5,
                                'xref': 'paper',
                                'y': -0.25,
                                'yref': 'paper'}],
               'template': '...',
               'title': {'font': {'size': 13},
                         'text': 'Genieten arme mensen evenveel van burgerlijke vrijheden als rijke mensen?'},
               'xaxis': {'title': {'text': 'Jaar'}},
               'yaxis': {'title': {'text': 'Score'}}}
})

## Analyse van stelling: Gelijkheid

De tweede stelling luidt: 'Genieten arme mensen evenveel van burgerlijke vrijheden als rijke mensen?'. IJsland weergeeft direct een interessante visualisatie. De score m.b.t. tot de stelling daalt aanzienlijk. Dit heeft dan weer geen gevolg op de happiness index. Dit blijkt echter niet uit de PCC van -0.77. 

De score m.b.t. de stelling van Venezuela fluctueert lichtelijk. De lichte daling loopt relatief gelijk met de sterkere daling van de happiness index van een 7 naar een 5. Dit blijkt ook uit de PCC van 0.83. 

Roemenië scoort relatief hoog m.b.t. de stelling, rond de 7.6. Vanaf 2017 gaat deze echter fluctueren. Er vindt een daling plaats, waarna deze weer redelijk wordt gecorrigeerd. De happiness index stijgt vanaf 2015.

In [8]:
display(container3, fig_widget3)


FigureWidget({
    'data': [],
    'layout': {'annotations': [{'showarrow': False,
                                'text': ('Deze Grafiek toont de relatie ' ... 'e mate van censuur in een land'),
                                'x': 0.5,
                                'xref': 'paper',
                                'y': -0.25,
                                'yref': 'paper'}],
               'template': '...',
               'title': {'font': {'size': 13},
                         'text': ('Probeert de regering op direct' ... 'of uitzendmedia te censureren?')},
               'xaxis': {'title': {'text': 'Jaar'}},
               'yaxis': {'title': {'text': 'Score'}}}
})

## Analyse van stelling: Censuur

De derde stelling luidt: 'Probeert de regering op directe of indirecte wijze de gedrukte of uitzendmedia te censureren?' In IJsland zien we een forse stijging in 2016-2017, los van deze stijging blijft de score stabiel. De happiness index wordt hier vrijwel niet door beinvloedt, wat ook blijkt uit de lage PCC van 0.33. 

Venezuela scoort relatief laag m.b.t. de stelling. Media in het land wordt dus veel gecensureerd. De score daalt gedurende de jaren, net als de happiness index. De PCC is dan ook 0.83. 

De score van Romenië fluctueert het meest. De scores wijken uit van 8.2 tot 5.5. De happiness index wordt hierdoor niet beïnvloed. Deze toont een gelijdelijke stijging.

# Conclusies

### Verband Aanwezig

Als we kijken naar de data over Venezuela, zouden we kunnen zeggen dat er zeker een verband is tussen de mate van democratie en ervaring van geluk. Als de happiness index afneemt, nemen de scores voor de drie stellingen onder de streep ook af. Soms vindt er nog een lichte stijging plaats, maar uiteindelijk eindigt deze altijd lager. De drie PCC’s m.b.t. Venezuela zeggen hetzelfde. Deze liggen alledrie rond de 0.80, wat zegt dat er een grote correlatie is tussen de twee datasets. In Venezuela gebeurt in het land de afgelopen tijd veel. Er is veel corruptie en inflatie. Dit heeft natuurlijk ook invloed op zowel de mate van democratie als de ervaring van geluk. Hierdoor wordt de conclusie dus nog meer bevestigd en hierdoor is deze data het meest waardevol voor dit perspectief.


### Verband Afwezig

Toch zou je ook kunnen stellen dat er geen verband is tussen de democracy index en de happiness index. Geluk en democratie zijn beide ingewikkelde, subjectieve maatstaven die lastig te meten zijn. Daarnaast kunnen er andere variabelen en invloeden zijn die van invloed zijn op geluk en democratie, maar niet in de gebruikte data zijn weergegeven. Sociale, economische, culturele en historische factoren kunnen allemaal een rol spelen bij het bepalen van het niveau van geluk en de staat van democratie in een land.

## Reflectie  

De feedback die we hebben ontvangen was een beetje verwarrend, voornamelijk omdat we op dat moment nog niet klaar waren met onze Draft. Daarom kon het groepje dat ons feedback gaf, ook geen volledig nauwkeurige feedback geven. Echter kregen we wel een aantal positieve aspecten over onze conceptversie te horen van zowel het groepje als de TA. De TA adviseerde ons verder om minder algemene visualisaties te gebruiken en meer onderbouwende visualisaties te presenteren (omtrent de voor en tegen argumenten). We hebben geprobeerd dit advies te reflecteren in de grafieken die hierboven staan.

## Werkverdeling

Bryan: Meeste van het coderen gedaan om dingen werkend te krijgen, heeft het fundament van de uiteindelijke code grotendeels geschreven. Kwam met het idee voor een wereldkaart als visualisatie en heeft deze zelf werkend gekregen, Schreef ook de reflectie.

Mike: Verantwoordelijk geweest voor het vinden van de uiteindelijk datasets, heeft de Democracy Indices dataset zelfstandig geanalyseerd en de stellingen gevonden voor analyse. Verantwoordelijk geweest voor het samenvoegen van de uiteindelijke onderdelen tot de ingeleverde Notebook. Schreef “Datasets and preprocessing”, “Creëren van de uiteindelijke dataset”, “Perspectieven” en “Observaties”.

Daniel: Mijn bijdrage aan onze data story is het helpen met vinden van onze datasets, het schrijven van de inleiding, het schrijven van de omschrijvingen van de figuren van de stellingen, en het schrijven van de conclusie. Ook hebben we met z’n drieeën veel gespard over onze datasets, observaties, en conclusies. In een werkcollege heb ik met behulp van een TA onze GitHub link werkend gekregen.